Setup

In [41]:
import docker
from pathlib import Path

docker_client = docker.from_env()
img = docker_client.images.pull(
    "ghcr.io/becksteinlab/streaming-md-docker:main"
)
# Run the simulation
# sudo apt-get install -y nvidia-container-toolkit
container = docker_client.containers.run(
            img, 
            f"/bin/sh -c 'cd /tmp && lmp < lammps_v3.in'",
            detach=True,
            volumes={"/Users/ljwoods2/workspace/imd-workshop-2024/lammps_demos": {"bind": "/tmp", "mode": "rw"}},
            ports={"8888/tcp": 8888},
            # runtime='nvidia'
            remove=True
        )

Analysis

ContainerError: Command '/bin/sh -c 'cd /tmp && lmp < lammps_v3.in'' in image 'ghcr.io/becksteinlab/streaming-md-docker:main' returned non-zero exit status 1: b'bash: cannot set terminal process group (-1): Inappropriate ioctl for device\nbash: no job control in this shell\n/bin/sh: lammps_v3.in: No such file or directory\n'



In [42]:
from imdclient.IMDREADER import IMDReader
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSF
from MDAnalysis.analysis.rdf import InterRDF
import logging


logger = logging.getLogger("imdclient.IMDClient")
file_handler = logging.FileHandler("gromacs_test.log")
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.setLevel(logging.DEBUG)


import concurrent.futures
u = mda.Universe("/Users/ljwoods2/workspace/imd-workshop-2024/lammps_demos/topology_after_min.data", "localhost:8888", atom_style="id type x y z",)
g = u.atoms

rdf = InterRDF(g, g)
rdf._ts = u.trajectory.ts

rmsf = RMSF(g)
rmsf._ts = u.trajectory.ts

rdf._prepare()
rmsf._prepare()

executor = concurrent.futures.ThreadPoolExecutor()

count = 0
for ts in u.trajectory:
    rsf_frame = executor.submit(rdf._single_frame)

    rmsf._frame_index = count
    rmsf_frame =executor.submit(rmsf._single_frame)

    # Wait for each thread to finish
    rsf_frame.result()
    rmsf_frame.result()

    count += 1

rdf.n_frames = count
rdf._conclude()

rmsf._conclude()

rdf.results["rdf"]
rmsf.results["rmsf"]


array([0.07832845, 0.06785146, 0.08970784, ..., 0.05451041, 0.08909698,
       0.05102232])